# Environment  
## Load required packages  


In [ ]:
library(Seurat)
library(tidyverse)
library(viridis)

In [ ]:
sample <- params$sample # sample_condition
condition <- str_split_1(sample, '_')[2]

## Loading data  
*min.cells* : Include features detected in at least this many cells. Will subset the counts matrix as well.  
*min.features* : Include cells where at least this many features are detected.  


In [ ]:
path <- paste0("../../../Processing/samples/", sample)

mtx_obj <- Seurat::ReadMtx(mtx = paste(path,"/matrix.mtx.gz",sep = ""), 
                           cells = paste(path,"/barcodes.tsv.gz",sep = ""),
                           features = paste(path,"/features.tsv.gz",sep = ""))

seurat_obj <- CreateSeuratObject(counts = mtx_obj,
                                        min.cells = 3, min.features = 200,
                                        project = sample)

seurat_obj@meta.data$condition <- condition     
seurat_obj    

In [ ]:
seurat_obj <- readRDS(file = paste0("./results/rds/mtx/mtx_", sample, ".rds"))
                
seurat_obj                  

# QC metrics (RAW)
**nCount_RNA** : the total number of reads (or more correctly UMIs) in the dataset  
**nFeature_RNA** : the number of observed genes (anything with a nonzero count)

##  <span style="color:#296d98;"> Parameters </span>  


In [ ]:
## nCount
x1 = 500
## nFeature
x2 = 220
## Mitochondrion
x3 = 15


## Nb of reads  


In [ ]:
ggplot(seurat_obj[[]]) + 
   geom_histogram(aes(nCount_RNA, 
                  color = ifelse(nCount_RNA < x1 , "a", "b")),
                  binwidth = 50) + 
  scale_color_manual(values = c("a" = "#558bdc", "b" = "#173664")) +
  theme_classic()+
  ggtitle("Distribution of nCount_RNA ") + NoLegend()



## Nb of genes  


In [ ]:
ggplot(seurat_obj[[]]) + 
   geom_histogram(aes(nFeature_RNA, 
                  color = ifelse(nFeature_RNA < x2 , "a", "b")),
                  binwidth = 50) + 
  scale_color_manual(values = c("a" = "#558bdc", "b" = "#173664")) +
  ggtitle("Distribution of nFeature_RNA ") + NoLegend()

Low-quality cells or empty droplets will often have very few genes.  
Cell doublets or multiplets may exhibit an aberrantly high gene count.

## MT genes  
Single cell datasets can be filled with large numbers of reads coming from mitochondria. These often indicate a sick cell undergoing apoptosis.
This metric can identify whether there is a large amount of mitochondrial contamination from dead or dying cells. We define poor quality samples for mitochondrial counts as cells which surpass the 20 mitochondrial ratio mark (unless it is expected in the sample).


In [ ]:
seurat_obj[["percent.mt"]] <- PercentageFeatureSet(seurat_obj, pattern = "^MT-")

ggplot(seurat_obj[[]]) + 
   geom_histogram(aes(percent.mt, 
                  color = ifelse(percent.mt < x3 , "a", "b")),
                  binwidth = 0.5) + 
  scale_color_manual(values = c("a" = "#173664", "b" = "#558bdc")) + 
  scale_x_continuous(breaks=seq(0, 100, by = 15))+
  ggtitle("Distribution of Percentage Mitochondrion") + NoLegend()


## Ribosomal genes  
Ribosomal genes also tend to be very highly represented, and can vary between cell types, so it can be instructive to see how prevalent they are in the data. These are ribosomal protein genes rather than the actual rRNA, so they’re more a measure of the translational activity of the cell rather than the cleanliness of the polyA selection.


In [ ]:
PercentageFeatureSet(seurat_obj,pattern="^RP[LS]") -> seurat_obj[["percent.ribosomal"]] 

ggplot(seurat_obj[[]]) + 
   geom_histogram(aes(percent.ribosomal), 
                  color = "#173664", fill= "#558bdc",
                  binwidth = 0.5) + 
  ggtitle("Distribution of Percentage Ribosomal") + NoLegend()


## Largest gene  
what percentage of the data comes from the single most observed gene?

Again, having a high proportion of your data dominated by a single gene is a metric which could either give biological context or indicate a technical problem, depending on what the gene is.


In [ ]:
apply(
  seurat_obj@assays$RNA@counts,
  2,
  max
) -> largest_count

apply(
  seurat_obj@assays$RNA@counts,
  2,
  which.max
) -> largest_index

rownames(seurat_obj)[largest_index] -> seurat_obj$largest_gene
100 * largest_count / seurat_obj$nCount_RNA -> seurat_obj$percent.largest_gene


In [ ]:
ggplot(seurat_obj[[]]) + 
   geom_histogram(aes(percent.largest_gene), 
                  color = "#173664", fill= "#558bdc",
                  binwidth = 0.7) + 
  ggtitle("Distribution of Percentage Largest Gene") + NoLegend()



## Plots  


In [ ]:
VlnPlot(seurat_obj, features=c("nFeature_RNA","nCount_RNA"), ncol = 2, pt.size = 0)
VlnPlot(seurat_obj, features=c("percent.mt", "percent.ribosomal", "percent.largest_gene"), ncol = 3, pt.size = 0)
FeatureScatter(seurat_obj, feature1 = "nCount_RNA", feature2 = "percent.mt")
FeatureScatter(seurat_obj,feature1 = "nCount_RNA", feature2 = "percent.largest_gene")


In [ ]:
FeatureScatter(seurat_obj, feature1 = "nCount_RNA", feature2 = "nFeature_RNA") + geom_smooth(method = 'lm')

A good datatset should follow the line.

# Filtering  
## Subset object


In [ ]:
as_tibble(
  seurat_obj[[]],
  rownames="Cell.Barcode"
) -> qc.metrics

raw_counts <- qc.metrics[, c("Cell.Barcode","nCount_RNA")]
head(qc.metrics)

In [ ]:
subset(
  seurat_obj,
    nCount_RNA> x1 & 
    nFeature_RNA > x2 & 
    percent.mt < x3 ) -> seurat_obj

seurat_obj



# Save seurat object (RAW)


In [ ]:
## Raw data
saveRDS(seurat_obj, file =paste0("./results/rds/samples/QC_",sample,".rds"))


## Doublets  
### Pre-process standard workflow  


In [ ]:
seurat_obj <- NormalizeData(seurat_obj)
seurat_obj <- FindVariableFeatures(seurat_obj)
seurat_obj <- ScaleData(seurat_obj)
seurat_obj <- RunPCA(seurat_obj)

ElbowPlot(seurat_obj)


# QC metrics (Mid-filtering)


In [ ]:
FeatureScatter(seurat_obj, feature1 = "nCount_RNA", feature2 = "nFeature_RNA") + geom_smooth(method = 'lm')

In [ ]:
as_tibble(
  seurat_obj[[]],
  rownames="Cell.Barcode"
) -> qc.metrics


## Nb of reads   


In [ ]:
ggplot(qc.metrics) + 
   geom_histogram(aes(nCount_RNA), 
                  color = "#558bdc", fill= "#173664",
                  binwidth = 50) + 
  ggtitle("Distribution of nCount_RNA ") + NoLegend()

In [ ]:
raw_counts$filtered <- NA
raw_counts[rownames(qc.metrics),"filtered"] <- qc.metrics[,"nCount_RNA"]
epsilon <- 1 # pseudo-count to avoid problems with log(0)
boxplot(log2(raw_counts[,c("filtered","nCount_RNA")] + epsilon), pch=".", 
        horizontal=TRUE, cex.axis=0.5,
        las=1, main="Raw counts (before & after filtering)", xlab="log2(Counts +1)", col=viridis(5, alpha=0.6, direction = 1))


## Nb of genes  


In [ ]:
ggplot(qc.metrics) + 
   geom_histogram(aes(nFeature_RNA), 
                  color = "#558bdc", fill= "#173664",
                  binwidth = 50) + 
  ggtitle("Distribution of nFeature_RNA ") + NoLegend()

## MT genes  


In [ ]:
ggplot(qc.metrics) + 
   geom_histogram(aes(percent.mt), 
                  color = "#558bdc", fill= "#173664",
                  binwidth = 0.5) +
   scale_x_continuous(breaks=seq(0, 100, by = 15))+
  ggtitle("Distribution of Percentage Mitochondrion") + NoLegend()


## Ribosomal genes  


In [ ]:
ggplot(qc.metrics) + 
   geom_histogram(aes(percent.ribosomal), 
                  color = "#558bdc", fill= "#173664",
                  binwidth = 0.5) + 
  ggtitle("Distribution of Percentage Ribosomal") + NoLegend()


## Largest gene  


In [ ]:
ggplot(qc.metrics) + 
   geom_histogram(aes(percent.largest_gene), 
                  color = "#558bdc", fill= "#173664",
                  binwidth = 0.7) + 
  ggtitle("Distribution of Percentage Largest Gene") + NoLegend()

 
# Save seurat object


In [ ]:
## Norm data
saveRDS(seurat_obj, file =paste0("./results/rds/samples/Doublet_",sample,".rds"))